In [ ]:
from pymadx.madx import Madx
import math

madx = Madx(stdout=False)

# Helper to send MAD-X commands
def run_block(lines):
    if isinstance(lines, list):
        cmd = "\n".join(lines)
    else:
        cmd = lines
    madx.input(cmd)

In [ ]:
universal_mass_unit = 0.931494061

initial_tune_x = 4.329
initial_tune_y = 3.29
final_tune_x = initial_tune_x
resonant_tune_x = 13.0 / 3.0

central_tune_x = 0.5 * (initial_tune_x + resonant_tune_x)
ko_width_factor = 1.5
bit_length_factor = abs(central_tune_x - resonant_tune_x) * ko_width_factor
bit_length_factor = 1.0 / (abs(bit_length_factor) + 1.0e-30)
bit_length_factor = math.floor(bit_length_factor)

while central_tune_x > 1.0:
    central_tune_x -= 1.0

ko_kick_amplitude = 2.0e-5

delta_tune_x = -1.0
delta_tune_y = -1.0

max_emit_x_ini = 150.0e-6
max_emit_y_ini = 50.0e-6
kinetic_energy_ini_GeV = 0.00676
gamma_rel_ini = (universal_mass_unit + kinetic_energy_ini_GeV) / universal_mass_unit
beta_rel_ini = math.sqrt(1.0 - 1.0 / gamma_rel_ini ** 2)

kinetic_energy_fin_GeV = 0.5
gamma_rel_fin = (universal_mass_unit + kinetic_energy_fin_GeV) / universal_mass_unit
beta_rel_fin = math.sqrt(1.0 - 1.0 / gamma_rel_fin ** 2)

max_emit_x_fin = max_emit_x_ini * beta_rel_ini * gamma_rel_ini / (beta_rel_fin * gamma_rel_fin)
max_emit_y_fin = max_emit_y_ini * beta_rel_ini * gamma_rel_ini / (beta_rel_fin * gamma_rel_fin)
rms_emit_x_fin = max_emit_x_fin / 4.0
rms_emit_y_fin = max_emit_y_fin / 4.0
dp_max = 5.0e-4

delta_tune_x /= beta_rel_fin
delta_tune_y /= beta_rel_fin

quad_str_GS01QS1F = 0.37611
quad_str_GS01QS2D = -0.35690
quad_str_GS12QS1F = quad_str_GS01QS1F
quad_str_GS12QS2D = quad_str_GS01QS2D
quad_str_GS12QS3T = 0.02989

quad_str_GS01QS1F_ref = quad_str_GS01QS1F
quad_str_GS01QS2D_ref = quad_str_GS01QS2D
quad_str_GS12QS3T_ref = quad_str_GS12QS3T

ion_mass_number = 40.0
ion_charge_state = 18.0
ion_mass_GeV = ion_mass_number * universal_mass_unit / ion_charge_state
ion_charge_state = 1.0 

ring_circumference = 216.72

In [ ]:
run_block(f"""
umass={universal_mass_unit};
qx={initial_tune_x};
qy={initial_tune_y};
qx_fin={final_tune_x};
qx_r={resonant_tune_x};
qx_c={central_tune_x};
kowidth_factor={ko_width_factor};
bit_len={bit_length_factor};
kokick_a={ko_kick_amplitude};

dqx={delta_tune_x};
dqy={delta_tune_y};

epsx_max_ini={max_emit_x_ini};
epsy_max_ini={max_emit_y_ini};
energy_ini={kinetic_energy_ini_GeV};
gamma_ini={gamma_rel_ini};
beta_ini={beta_rel_ini};

energy_fin={kinetic_energy_fin_GeV};

gamma_fin={gamma_rel_fin};
beta_fin={beta_rel_fin};
epsx_max_fin={max_emit_x_fin};
epsy_max_fin={max_emit_y_fin};
epsx_rms_fin={rms_emit_x_fin};
epsy_rms_fin={rms_emit_y_fin};
dpmax={dp_max};

k1nl_GS01QS1F:={quad_str_GS01QS1F};
k1nl_GS01QS2D:={quad_str_GS01QS2D};
k1nl_GS12QS1F:={quad_str_GS12QS1F};
k1nl_GS12QS2D:={quad_str_GS12QS2D};
k1nl_GS12QS3T:={quad_str_GS12QS3T};

k2_S01KS1C=0.0; k2_S01KS3C=0.0; k2_S03KS1C=0.0; k2_S03KS3C=0.0; k2_S05KS1C=0.0;
k2_S05KS3C=0.0; k2_S07KS1C=0.0; k2_S07KS3C=0.0; k2_S09KS1C=0.0; k2_S09KS3C=0.0;
k2_S11KS1C=0.0; k2_S11KS3C=0.0; k2_ks1c=0.0; k2_ks3c=0.0;

call, file=phi_ressext.madx;
call, file="SIS18RING.SEQ";
call, file=insert_elem_def_sis18.madx;

vrf=0.0;
harmonic_number=5;
v1=vrf;

circ={ring_circumference};

beam, mass={ion_mass_GeV}/{ion_charge_state},gamma={gamma_rel_fin},charge={ion_charge_state};
use, sequence=sis18ring;

select, flag=twiss, clear;
select, flag=twiss, column=name,keyword,s,l;
twiss, table=twiss, file=sis18ring_thick0.twiss;
twiss, save;
""")

In [ ]:
run_block([
    'seqedit, sequence=sis18ring;',
    'flatten;',
    'cycle, start=GS04ME1E_en;',
    'flatten;',
    'endedit;'
])

run_block(["use, sequence=sis18ring;",
           'select, flag=twiss, clear;',
           'select, flag=twiss, column=name,keyword,s,l;',
           'twiss, table=twiss, file=sis18ring_thick1.twiss;',
           'twiss, save;'])

run_block(['save, sequence=sis18ring, file=sis18ring_thick.seq;', 'call, file=file=sis18ring_thick.seq;'])

run_block([
    'seqedit, sequence=sis18ring;',
    'flatten;',
    'replace, element=GS04ME1E_en, by=S04ME1EE1;',
    'replace, element=GS04ME1E_ex, by=S04ME1EE2;',
    'replace, element=GS06MU3E_en, by=s06mu3e1;',
    'replace, element=GS06MU3E_ex, by=s06mu3e2;',
    'flatten;',
    'endedit;'
])

run_block(['use, sequence=sis18ring;',
           'select, flag=twiss, clear;',
           'select, flag=twiss, column=name,keyword,s,l;',
           'twiss, table=twiss, file=sis18ring_thick3.twiss;',
           'twiss, save;'])

run_block([
    'select,flag=makethin,sequence=sis18ring,class=sbend,slice=5;',
    'select,flag=makethin,sequence=sis18ring,class=quad_long,slice=10;',
    'select,flag=makethin,sequence=sis18ring,class=quad_short,slice=10;',
    'makethin, sequence=sis18ring;'
])

run_block(['use, sequence=sis18ring;',
           'select, flag=twiss, clear;',
           'select, flag=twiss, column=name,keyword,s,l;',
           'twiss, table=twiss, file=sis18ring_thin0.twiss;',
           'twiss, save;'])

install_lines = [
    'seqedit, sequence=sis18ring;',
    'flatten;'
]
install_lines += [
    'install, element=fastquad, at=0.0, from=GS02KQ1E;',
    'install, element=fastquad, at=0.0, from=GS08KQ1E;',
    'install, element=rfcav1, at=0.0, from=GS02BE1A;',
    'replace, element=GS01BO1EH, by=kokicker;'
]

qs_pairs = [
    ('GS01QS1F_T','GS01QS1F','GS12QS1F_T','GS02QS1F','GS01QS1F_T','GS03QS1F','GS12QS1F_T','GS04QS1F'),
]

In [ ]:
install_lines += [
    'install, element=GS01QS1F_T,at=0.0, from=GS01QS1F;',
    'install, element=GS12QS1F_T,at=0.0, from=GS02QS1F;',
    'install, element=GS01QS1F_T,at=0.0, from=GS03QS1F;',
    'install, element=GS12QS1F_T,at=0.0, from=GS04QS1F;',
    'install, element=GS01QS1F_T,at=0.0, from=GS05QS1F;',
    'install, element=GS12QS1F_T,at=0.0, from=GS06QS1F;',
    'install, element=GS01QS1F_T,at=0.0, from=GS07QS1F;',
    'install, element=GS12QS1F_T,at=0.0, from=GS08QS1F;',
    'install, element=GS01QS1F_T,at=0.0, from=GS09QS1F;',
    'install, element=GS12QS1F_T,at=0.0, from=GS10QS1F;',
    'install, element=GS01QS1F_T,at=0.0, from=GS11QS1F;',
    'install, element=GS12QS1F_T,at=0.0, from=GS12QS1F;',
    'install, element=GS01QS2D_T,at=0.0, from=GS01QS2D;',
    'install, element=GS01QS2D_T,at=0.0, from=GS02QS2D;',
    'install, element=GS01QS2D_T,at=0.0, from=GS03QS2D;',
    'install, element=GS01QS2D_T,at=0.0, from=GS04QS2D;',
    'install, element=GS01QS2D_T,at=0.0, from=GS05QS2D;',
    'install, element=GS01QS2D_T,at=0.0, from=GS06QS2D;',
    'install, element=GS01QS2D_T,at=0.0, from=GS07QS2D;',
    'install, element=GS01QS2D_T,at=0.0, from=GS08QS2D;',
    'install, element=GS01QS2D_T,at=0.0, from=GS09QS2D;',
    'install, element=GS01QS2D_T,at=0.0, from=GS10QS2D;',
    'install, element=GS01QS2D_T,at=0.0, from=GS11QS2D;',
    'install, element=GS01QS2D_T,at=0.0, from=GS12QS2D;',
    'install, element=GS12QS3T_T,at=0.0, from=GS01QS3T;',
    'install, element=GS12QS2T_T,at=0.0, from=GS02QS3T;',
    'install, element=GS12QS3T_T,at=0.0, from=GS03QS3T;',
    'install, element=GS12QS3T_T,at=0.0, from=GS04QS3T;',
    'install, element=GS12QS3T_T,at=0.0, from=GS05QS3T;',
    'install, element=GS12QS3T_T,at=0.0, from=GS06QS3T;',
    'install, element=GS12QS3T_T,at=0.0, from=GS07QS3T;',
    'install, element=GS12QS3T_T,at=0.0, from=GS08QS3T;',
    'install, element=GS12QS3T_T,at=0.0, from=GS09QS3T;',
    'install, element=GS12QS3T_T,at=0.0, from=GS10QS3T;',
    'install, element=GS12QS3T_T,at=0.0, from=GS11QS3T;',
    'install, element=GS12QS3T_T,at=0.0, from=GS12QS3T;'
]

install_lines += ['flatten;', 'endedit;']
run_block(install_lines)

run_block(['use, sequence=sis18ring;',
           'select, flag=twiss, clear;',
           'select, flag=twiss, column=name,keyword,s,l;',
           'twiss, table=twiss, file=sis18ring_thin0.twiss;',
           'twiss, save;'])

run_block([
    'match,SEQUENCE=sis18ring;',
    'VARY,NAME=k1nl_GS01QS1F,STEP=0.001;!,LOWER=_0.4, UPPER=_0.10;',
    'VARY,NAME=k1nl_GS01QS2D,STEP=0.001;!,LOWER= 0.20, UPPER= 0.4;',
    'CONSTRAINT,SEQUENCE=sis18ring,RANGE=#E,MUX=qx,MUY=qy;',
    'LMDIF,calls=20000,tolerance=1.e-8;',
    'endmatch;'
])

run_block(['select, flag=twiss, clear;', 'select, flag=twiss;', 'twiss, table=twiss;', 'twiss, save;'])

run_block([
    'match,SEQUENCE=sis18ring;',
    'VARY,NAME=k2_ks1c,STEP=0.001;!,LOWER=_0.4, UPPER=_0.10;',
    'VARY,NAME=k2_ks3c,STEP=0.001;!,LOWER= 0.20, UPPER= 0.4;',
    'GLOBAL, SEQUENCE=sis18ring, DQ1=DQX;',
    'GLOBAL, SEQUENCE=sis18ring, DQ2=DQy;',
    'LMDIF,calls=20000,tolerance=1.e-8;',
    'endmatch;'
])

run_block(['select, flag=twiss, clear;', 'select, flag=twiss, column=name,keyword,s,l;', 'twiss, table=twiss, file=sis18ring_thin1.twiss;', 'twiss, save;'])

run_block([
    'match,SEQUENCE=sis18ring;',
    'VARY,NAME=k1l_fast,STEP=0.001;!,LOWER= 0.20, UPPER= 0.4;',
    f'CONSTRAINT,SEQUENCE=sis18ring,RANGE=#E,MUX={qx_fin};',
    'LMDIF,calls=20000,tolerance=1.e-8;',
    'endmatch;'
])

run_block(['select, flag=twiss, clear;', 'select, flag=twiss, column=name,keyword,s,l;', 'twiss, table=twiss, file=sis18ring_thin2.twiss;', 'twiss, save;'])

run_block(['k1l_fast_max=k1l_fast;', 'k1l_fast=0.0;'])

run_block(['select, flag=twiss, clear;', 'select, flag=twiss;', 'twiss, table=twiss;', 'twiss, save;'])

In [ ]:
essep_bump = 0.018
k0nl_GS04MU1 = -0.115 * essep_bump
k0nl_GS05MU2 = -0.104 * essep_bump
mgsep_bump = 0.006
k0nl_GS06MU1 = 0.268 * mgsep_bump
k0nl_GS07MU2 = 0.242 * mgsep_bump

run_block([
    f'k0nl_GS04MU1:={k0nl_GS04MU1};',
    f'k0nl_GS05MU2:={k0nl_GS05MU2};',
    f'k0nl_GS06MU1:={k0nl_GS06MU1};',
    f'k0nl_GS07MU2:={k0nl_GS07MU2};'
])

run_block(['call, file="SIS18_cryocatchers.str";',
           'select, flag=error, clear;',
           'select, flag=error,pattern="S04ME1EE1";',
           'select, flag=error,pattern="S04ME1EE2";',
           'ealign, arex=0.1;'])

run_block([
    'k2_S01KS1C:=k2la*sin(0.0/6.0*twopi+phi);',
    'k2_S03KS1C:=k2la*sin(1.0/6.0*twopi+phi);',
    'k2_S05KS1C:=k2la*sin(2.0/6.0*twopi+phi);',
    'k2_S07KS1C:=k2la*sin(3.0/6.0*twopi+phi);',
    'k2_S09KS1C:=k2la*sin(4.0/6.0*twopi+phi);',
    'k2_S11KS1C:=k2la*sin(5.0/6.0*twopi+phi);'
])

run_block([
    'select, flag=twiss, clear;',
    'select, flag=twiss, column=s,x,px,y,py;',
    'twiss, table=twiss, file=sis18ring_cod.twiss;',
    'twiss, save;',
    'select, flag=twiss, clear;',
    'select, flag=twiss, column=name,keyword,s,x,px,y,py,betx,alfx,dx,dpx,mux,bety,alfy,dy,dpy,muy,k1l,k2l;',
    'twiss, table=twiss, file=sis18ring_cod_withnames.twiss;',
    'twiss, save;',
    'select, flag=twiss, clear;',
    'select, flag=twiss, column=name,keyword,s,l,x,px;',
    'twiss, table=twiss, file=sis18ring_thin3.twiss;',
    'twiss, save;'
])

run_block([
    'circ=table(summ,length);',
    'alfx0=table(twiss,sis18ring$start,alfx);',
    'betx0=table(twiss,sis18ring$start,betx);',
    'alfy0=table(twiss,sis18ring$start,alfy);',
    'bety0=table(twiss,sis18ring$start,bety);',
    'gamx0=(1.0+alfx0*alfx0)/betx0;',
    'gamy0=(1.0+alfy0*alfy0)/bety0;',
    'xco=table(twiss,sis18ring$start,dx);',
    'pxco=table(twiss,sis18ring$start,dpx);',
    'yco=table(twiss,sis18ring$start,dy);',
    'pyco=table(twiss,sis18ring$start,dpy);',
    'xco0=table(twiss,sis18ring$start,x);',
    'pxco0=table(twiss,sis18ring$start,px);',
    'yco0=table(twiss,sis18ring$start,y);',
    'pyco0=table(twiss,sis18ring$start,py);',
    'tco0=table(twiss,sis18ring$start,t);',
    'ptco0=table(twiss,sis18ring$start,pt);',
    'xco2=table(twiss,sis18ring$start,ddx);',
    'pxco2=table(twiss,sis18ring$start,ddpx);',
    'yco2=table(twiss,sis18ring$start,ddy);',
    'pyco2=table(twiss,sis18ring$start,ddpy);',
    'ct_sfp_co=tco0;',
    'pt_sfp_co=ptco0;'
])

In [ ]:
import numpy as np
import math
import time
import sixtracklib as stl

nptot = 100000               
rev_rf_ramp = 1000            
revmax = 10000                
revtot = revmax + rev_rf_ramp
bit_len = 10

rng = np.random.default_rng(123)

betx0 = 12.0
bety0 = 8.0
alfx0 = 0.0
alfy0 = 0.0
gamx0 = (1.0 + alfx0**2) / betx0
gamy0 = (1.0 + alfy0**2) / bety0
xco0 = 0.0
pxco0 = 0.0
yco0 = 0.0
pyco0 = 0.0
xco = 0.0
pxco = 0.0
yco = 0.0
pyco = 0.0
tco0 = 0.0
ptco0 = 0.0

epsx_rms_fin = 150.0e-6 / 4.0  
epsy_rms_fin = 50.0e-6 / 4.0
dpmax = 5.0e-4
beta_fin = 0.9  
circ = 216.72
c_light = 299792458.0
t_rev = circ / (c_light * beta_fin)

In [ ]:
kokick_a = 2.0e-5  
qx_c = 0.5 * (4.329 + 13.0/3.0)  

while qx_c > 1.0:
    qx_c -= 1.0


f_osc = 6.0e2
t_osc = 1.0 / f_osc
period_osc = t_osc / t_rev
phase = 0.0
sin_phase = math.sin(phase)
cos_phase = math.cos(phase)

k1nl_GS01QS1F_REL_a = 0.0
k1nl_GS01QS2D_REL_a = 0.0

n_bit_max = int(np.floor(1.5 * revmax / bit_len) + 1)
phi_ko_table = np.zeros(n_bit_max, dtype=float)
for ib in range(n_bit_max):
    v = rng.random()
    phi_ko_table[ib] = 0.0 if v < 0.5 else math.pi

np.savetxt("tab_phi_ko.dat", phi_ko_table, fmt="%.6e", header="phi_ko")

In [ ]:
randnum_av = rng.standard_normal(revmax + rev_rf_ramp + 10)  # a little margin
np.savetxt("randnum_av.dat", randnum_av, fmt="%.6e", header="randnum_av")

xrms = math.sqrt(epsx_rms_fin * betx0)
pxrms = math.sqrt(epsx_rms_fin / betx0)
yrms = math.sqrt(epsy_rms_fin * bety0)
pyrms = math.sqrt(epsy_rms_fin / bety0)
ptrms = 0.5 * dpmax * beta_fin

elems = stl.Elements()
elems.append_drift(1.0)
elems.append_drift(1.0)

try:
    kicker_index = elems.append_kick(0.0, 0.0)  # kick_x=0, kick_y=0 initial
    using_append_kick = True
except Exception:
    kicker_index = elems.append_multipole(0.0, knl=[0.0], ksl=[0.0])
    using_append_kick = False

mp_idx_1 = elems.append_multipole(0.0, knl=[0.0, 0.0, 0.0, 0.0])  # knl[0]=k0, knl[1]=k1, knl[2]=k2, knl[3]=k3
mp_idx_2 = elems.append_multipole(0.0, knl=[0.0, 0.0, 0.0, 0.0])
mp_idx_3 = elems.append_multipole(0.0, knl=[0.0, 0.0, 0.0, 0.0])

elems.append_drift(1.0)

In [ ]:
max_tries = int(10 * nptot)
particles_list = []

np_count = 0
tries = 0
np_min = 0  

t_min_max_av = 1.0  
while np_count < nptot and tries < max_tries:
    tries += 1

    xini = xrms * rng.normal()
    pxini = pxrms * rng.normal()
    pxini = pxini - alfx0 / betx0 * xini

    yini = yrms * rng.normal()
    pyini = pyrms * rng.normal()
    pyini = pyini - alfy0 / bety0 * yini

    xini += xco0
    pxini += pxco0
    yini += yco0
    pyini += pyco0

    epsx = betx0 * (pxini - pxco0) ** 2 + 2.0 * alfx0 * (xini - xco0) * (pxini - pxco0) + gamx0 * (xini - xco0) ** 2
    epsy = bety0 * (pyini - pyco0) ** 2 + 2.0 * alfy0 * (yini - yco0) * (pyini - pyco0) + gamy0 * (yini - yco0) ** 2

    tini = circ * (0.5 - rng.random()) / beta_fin / 5.0    
    ptini = ptrms * rng.normal()

    xini += xco * ptini
    pxini += pxco * ptini
    yini += yco * ptini
    pyini += pyco * ptini
    ptini += ptco0

    eps_switch = epsx / (epsx_rms_fin * 4.0) + epsy / (epsy_rms_fin * 4.0)  # adapt normalization
    long_switch = (tini / t_min_max_av) ** 2 + (ptini / (2.0 * ptrms + 1e-30)) ** 2

    if eps_switch < 1.0 and long_switch < 1.0:
        particles_list.append((xini, pxini, yini, pyini, tini, ptini))
        np_count += 1

particles_np = np.array(particles_list, dtype=float) 
print(f"Generated {particles_np.shape[0]} particles (attempts {tries})")

In [ ]:
tracker = stl.Tracker(elements=elems)
parts = stl.ParticlesSet(len(particles_np))

parts.x = particles_np[:, 0].astype(np.float64)
parts.px = particles_np[:, 1].astype(np.float64)
parts.y = particles_np[:, 2].astype(np.float64)
parts.py = particles_np[:, 3].astype(np.float64)

try:
    parts.ct = particles_np[:, 4].astype(np.float64)
    parts.delta = particles_np[:, 5].astype(np.float64)
except Exception:
    parts.t = particles_np[:, 4].astype(np.float64)
    parts.ptau = particles_np[:, 5].astype(np.float64)

try:
    parts.update_from_host()
except Exception:
    pass

In [ ]:
k1l_fast_max = 0.0  
k2la_max = 0.0      
k2la = 0.0

tt = 0
n_bit = 0
phi0 = phi_ko_table[0]
kophase = 0.0

record_every = max(1, revmax // 100)  
kick_history = []
k2la_history = []

start_time = time.time()
for turn in range(1, revtot + 1):
    rev = turn - rev_rf_ramp
    if rev > 0:
        tt += 1
        if tt >= bit_len:
            tt = 0
            n_bit += 1
            if n_bit >= len(phi_ko_table):
                n_bit = len(phi_ko_table) - 1
            phi0 = phi_ko_table[n_bit]
        kophase += qx_c * 2.0 * math.pi
        kokick_t = kokick_a * math.sin(kophase + phi0)

        k1l_fast = k1l_fast_max * min(max(rev / float(revmax), 0.0), 1.0)
    else:
        v1 = 0.0 * (rev + rev_rf_ramp) / rev_rf_ramp
        k2la = k2la_max * (rev + rev_rf_ramp) / rev_rf_ramp
        kokick_t = 0.0

    idx_rev = max(0, min(len(randnum_av) - 1, max(0, rev)))
    signal_rand = randnum_av[idx_rev]
    signal_sin_n = 0.0
    signal_cos_n = 0.0

    norm = 1.0
    signal_sin_n /= norm
    signal_cos_n /= norm
    signal_rand /= norm

    k1nl_GS01QS2D_REL_t = k1nl_GS01QS2D_REL_a * (signal_sin_n + signal_rand)
    k1nl_GS01QS1F_REL_t = k1nl_GS01QS1F_REL_a * (signal_sin_n * cos_phase + signal_cos_n * sin_phase + signal_rand)
    k1nl_GS12QS1F_REL_t = k1nl_GS01QS1F_REL_t

    if using_append_kick:
        try:
            elems.set_kick(kicker_index, kokick_t, 0.0)   
        except Exception:
            try:
                elems._data['knl'][kicker_index][0] = kokick_t
            except Exception:
                pass
    else:
        try:
            knl = elems.get_knl(mp_idx_1) 
            knl[0] = kokick_t
            elems.set_knl(mp_idx_1, knl)
        except Exception:
            try:
                elems._data['knl'][kicker_index][0] = kokick_t
            except Exception:
                pass

    for idx, relval in ((mp_idx_1, k1nl_GS01QS1F_REL_t),
                        (mp_idx_2, k1nl_GS12QS1F_REL_t),
                        (mp_idx_3, k1nl_GS01QS2D_REL_t)):
        try:
            knl = elems.get_knl(idx)     
            if len(knl) < 4:
                knl = list(knl) + [0.0] * (4 - len(knl))
            knl[3] = relval
            elems.set_knl(idx, knl)     
        except Exception:
            try:
                elems._data['knl'][idx][3] = relval
            except Exception:
                pass

    if turn % record_every == 0:
        kick_history.append((turn, kokick_t, k1nl_GS01QS1F_REL_t, k1nl_GS01QS2D_REL_t))
        k2la_history.append((turn, k2la))

    tracker.track(parts, 1)

elapsed = time.time() - start_time
print(f"Tracking completed: {revtot} turns, time {elapsed:.1f} s")
np.savetxt("kick_history.csv", np.array(kick_history), fmt="%.6e",
           header="turn,kick,k1_rel,k2_rel")
np.savetxt("k2la_history.csv", np.array(k2la_history), fmt="%.6e", header="turn,k2la")

try:
    parts.update_to_host()  
except Exception:
    pass

try:
    xs = parts.x.copy()
    pxs = parts.px.copy()
    ys = parts.y.copy()
    pys = parts.py.copy()
    cts = getattr(parts, "ct", getattr(parts, "t", np.zeros_like(xs))).copy()
    deltas = getattr(parts, "delta", getattr(parts, "ptau", np.zeros_like(xs))).copy()
except Exception:
    xs = particles_np[:, 0]
    pxs = particles_np[:, 1]
    ys = particles_np[:, 2]
    pys = particles_np[:, 3]
    cts = particles_np[:, 4]
    deltas = particles_np[:, 5]

out = np.vstack([xs, pxs, ys, pys, cts, deltas]).T
np.savetxt("part_coord_ini.csv", out, fmt="%.12e",
           header="x,px,y,py,ct,delta")

print("Wrote part_coord_ini.csv, kick_history.csv, k2la_history.csv")